第二回 tensor型ってなに？

今回からPytorchの少し細かい部分に首を突っ込んでいきます。まず、最初は前回出てきた「Pytorch専用配列」こと、tensor型についてです。

１、tensor型のコンセプト

tensor型とは、Pytorchの計算で用いられる型のことであり、一見、int型やfloat型と全く同じような性質を持ってるように思えますが、実は違うものです。

このtensor型は、Pytorchで作ったNNのパラメータ(重みやバイアス)、画像データやテキストデータの表現に使われ、並列計算をより効率的に行えるよう制作されています。とりあえず例を見ていきましょう。

In [1]:
import torch #tensor型は、torchライブラリを読み込まないと使えない

data=10

#tensor型の変数は、torchの中のtensorクラスのインスタンス
tensor_data=torch.tensor(data)

print(data)
print(tensor_data)

10
tensor(10)


このように、print()関数で表示したとき、tensor型のデータは通常の整数型とは異なり、「tensor()」という文字が出てきて表示されます。これは、「tensor_data」と呼ばれる変数が、tensor型であることを意味しています。

ただこれだけだと「え？ただの数値じゃね?」って思うかもしれないんですが、次の例を見ていただけると分かります。

次のコードは、あるサイズが同じ配列どうしで、添え字(index)が等しい要素同士の和を求め、同じサイズの配列として出力を行うというコードです。（配列同士の足し算ですね）

In [2]:
#なんか違和感のある代入の仕方ですが、これはアンパック代入と言います
array_1,array_2=[1,2,3,4],[5,6,7,8]

#足し合わせた結果を入れるための配列
sum_array=[]

for i in range(0,4):
#appendは配列に新しい要素を足し合わせるためのメソッドです。
    sum_array.append(array_1[i]+array_2[i])
#ここでは、array_1,2のi番目の要素を足し合わせてします。

print(sum_array)

[6, 8, 10, 12]


通常のpythonコードの場合、この配列同士の和であっても、for文を用いて求める必要があるので、正直どちゃくそめんどいです。しかし、tensor型は、この手間を解消してくれます。次のコードは、tensor型の配列同士の足し算をしています。

In [3]:
import torch

#これはアンパック代入と呼ばれる代入の仕方ですが、pythonでも普通にできます
tensor_array1,tensor_array2=torch.tensor([1,2,3,4]),torch.tensor([5,6,7,8])

#配列同士足しちゃってるぞ?! これはいったい、どうなっちゃうんだー!!?
sum_tensor=tensor_array1+tensor_array2

print(sum_tensor)

tensor([ 6,  8, 10, 12])


このように、tensor型の配列は、for文なんて使わなくても勝手に要素同士の和を求めてくれます。この機能のことを「ブロードキャスト」といいます。

このブロードキャストは、pytorch以外にも、numpyというライブラリで利用することができますがこの両者には圧倒的な違いが存在します。

それは、「この機能をGPUで使えるか、否か」です。pytorchのtensor型は、GPUでもこのブロードキャスト演算を行えますが、numpyは不可能なんです。深層学習ではこういったブロードキャストの処理を素早く行う必要があり、GPUの並列処理能力が不可欠なんです。だからこそ、pytorchではデータをtensor型にする必要があるのです。

2、tensor型の演算

では、このブロードキャスト機能を用いたpytorchによる演算を見ていきましょう。
あ、ついでなんですけど、次からする計算は2年生の「線形代数」っていう教科でよく出てくるのでもしかしたら授業の手助けになるかもです。

In [3]:
#その１、ベクトルと、行列

#tensor型のベクトルの生成
vec1=torch.tensor([1,2,3,4])

vec2=torch.tensor([4,5,6,7])

#tesor型の行列の生成
mat1=torch.tensor(
    [[0,1,2],
    [10,11,12]]
)

mat2=torch.tensor(
    [[3,4,5],
    [13,14,15]]
)

pytorchでは、このように「tensor()」の中に配列（二次元配列）を挿入することで、ベクトル、行列を作ることができます。

ここで、ベクトルと行列って言葉が出てきました。1年生（次の2年生）の方向けに解説しておきます。

ベクトル...大きさだけでなく、向きを持った量のこと。ここでは、配列のように複数のデータを持ったデータ構造として捉える。

行列...文字通り、数字や文字などを縦と横で並べたもの。2次元配列のような、ベクトルのベクトルというふうに捉えておこう。行列で表されたデータの代表としては「グレースケール画像」がある。

In [5]:
#その２、ベクトル、行列とただの数値の四則演算

#ベクトル
print(vec1+10)
print(vec1-5)
print(vec1*4)
print(vec1/10) #ここが大事

#行列
print(mat1+10)
print(mat1-5)
print(mat1*4)
print(mat1/10)

tensor([11, 12, 13, 14])
tensor([-4, -3, -2, -1])
tensor([ 4,  8, 12, 16])
tensor([0.1000, 0.2000, 0.3000, 0.4000])
tensor([[10, 11, 12],
        [20, 21, 22]])
tensor([[-5, -4, -3],
        [ 5,  6,  7]])
tensor([[ 0,  4,  8],
        [40, 44, 48]])
tensor([[0.0000, 0.1000, 0.2000],
        [1.0000, 1.1000, 1.2000]])


このように、ただの数値とベクトル、行列の四則演算では、全ての要素に対して、ただの数値が足されたり、かけられたりしていますね。これもブロードキャストの能力の一つです。

ついでに言うと、ベクトル、行列とは異なる「ただの数値」のことを、「スカラー量」と言います。補足ですが、線形代数ではスカラー量とベクトル・行列の和差なんて定義されていないのでPytorchオンリーの計算であると思って下さい。

さっきのコードで「ここが大事」って書いてあるところのポイントを解説します。
まずは、一体何が大事なのかという点ですね。ここで大事なのは、tensor型であっても、数値の型の変換自体は自動で行われるということです。（？）

次のコードで説明していきます。

In [6]:
#補足
print(vec1.dtype) #このdtypeは、torch.tensor型の属性(クラス自身がもつ変数みたいなもの)の一つです。
print((vec1/10).dtype)#このdtypeには、tensor型がもつデータの型情報が代入されています。

torch.int64
torch.float32


pytorchのtensor型と言っても、その中でも複数個の型に分けられています。この場合ですと、整数はtorch.int64、小数を含むデータの場合はtorch.float32となっております。このようにして、元はintのtensor型であっても、演算によってfloatのデータに変えてくれるということなんですね。ブロードキャスト恐るべし。

In [7]:
#その３、ベクトル or 行列　同士の四則演算

vec1=torch.tensor([1,2,3,4])
vec2=torch.tensor([4,5,6,7])
vec3=torch.tensor([6,7,8])

mat1=torch.tensor(
    [[0,1,2],
    [10,11,12]]
)

mat2=torch.tensor(
    [[3,4,5],
    [13,14,15]]
)

#ベクトルとベクトル
print(vec1+vec2)
print(vec1-vec2)
print(vec1*vec2)
print(vec1/vec2)

#行列とベクトル
print(mat1+vec3)
print(mat1-vec3)
print(mat1*vec3)
print(mat1/vec3)

#行列と行列
print(mat1+mat2)
print(mat1-mat2)
print(mat1*mat2)
print(mat1/mat2)

tensor([ 5,  7,  9, 11])
tensor([-3, -3, -3, -3])
tensor([ 4, 10, 18, 28])
tensor([0.2500, 0.4000, 0.5000, 0.5714])
tensor([[ 6,  8, 10],
        [16, 18, 20]])
tensor([[-6, -6, -6],
        [ 4,  4,  4]])
tensor([[ 0,  7, 16],
        [60, 77, 96]])
tensor([[0.0000, 0.1429, 0.2500],
        [1.6667, 1.5714, 1.5000]])
tensor([[ 3,  5,  7],
        [23, 25, 27]])
tensor([[-3, -3, -3],
        [-3, -3, -3]])
tensor([[  0,   4,  10],
        [130, 154, 180]])
tensor([[0.0000, 0.2500, 0.4000],
        [0.7692, 0.7857, 0.8000]])


ブロードキャストの力によって、各要素同士での演算が可能になっています。行列とベクトルの演算は特別で、行列が含む各ベクトルと、ベクトル同士の演算で表されています。

しかし、2、3年生(次の3,4年生)にとったら、少し違和感のある箇所があります。それは、掛け算のところです。本来の行列・ベクトルの掛け算は、もっと異なる形で定義されているはずですが、Pytorchでは要素同士の掛け算で表されています。

さらに驚くべき点は割り算です。通常の行列同士の割り算なんて定義されていなかったはずです。そのため、一般的な数学と、Pytorchの行列・ベクトルの乗除についてはっきり区別しておく必要があります。

このようなPytorchにおける行列・ベクトル同士の乗除を「アダマール積」といいます。1年生(次の2年生)にとったら残念ですが、今回は一般数学における行列・ベクトルの乗除については触れることはありません。

In [8]:
#その４、転置

mat3=torch.tensor(
    [[3,2,1],
    [6,5,4],
    [9,8,7]]
)

print(mat3)
print(mat3.T)

tensor([[3, 2, 1],
        [6, 5, 4],
        [9, 8, 7]])
tensor([[3, 6, 9],
        [2, 5, 8],
        [1, 4, 7]])


転置とは、行列の行と列を逆にする事です。つまり、i行のj列にある要素を、j行のi列にもっていくという事です。分かりにくいと思うので、コードで示します。

In [9]:
mat4=[[3,2,1],
    [6,5,4],
    [9,8,7]]

#いったん空の行列を作る
mat5=[[0,0,0],
    [0,0,0],
    [0,0,0]]

for i in range(0,3):
    for j in range(0,3):
        mat5[j][i]=mat4[i][j] #ここ見るとよくわかる

print(mat5) 

[[3, 6, 9], [2, 5, 8], [1, 4, 7]]


3、torchのtensor生成と、加工

Pytorchには、tensor型のデータを生成する方法がいくつかあります。１つは、これまでのように、自分たちで定義した配列を代入してtensor型にする方法です。そして2つ目として、torchのモジュールを用いる方法があります。

以下は、torchのモジュールによって、tensor型の行列・ベクトルを生成しています。

In [9]:
#これが、tensor行列や、ベクトルのサイズをしめす引数となる
shape1=(3)
shape2=(4,5)

#ベクトルの生成
rand_vec=torch.rand(shape1) #ランダムな値
one_vec=torch.ones(shape1) #全て1
zero_vec=torch.zeros(shape1) #全て0

print(rand_vec)
print(one_vec)
print(zero_vec)

#行列の生成
rand_mat=torch.rand(shape2)
one_mat=torch.ones(shape2)
zero_mat=torch.zeros(shape2)

print(rand_mat)
print(one_mat)
print(zero_mat)

tensor([0.5322, 0.4569, 0.4682])
tensor([1., 1., 1.])
tensor([0., 0., 0.])
tensor([[0.1922, 0.6777, 0.2448, 0.7856, 0.9380],
        [0.4102, 0.4485, 0.5694, 0.5236, 0.1905],
        [0.3857, 0.4275, 0.5215, 0.8969, 0.8357],
        [0.8670, 0.1209, 0.2396, 0.5318, 0.2857]])
tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


このようにして、pytorchはある一定のパターンを持つ行列を簡単に生成することができます。

また、tensor型のベクトル・行列は、配列と同じように扱える場面があり、ある要素だけ値を変更するといったことも可能です。ここでは、Pythonのスライス機能等を用いて、値を変化させてみます。

In [10]:
shape=(4,4)

tensor=torch.ones(shape)

print(tensor)

print(tensor[0]) #4*4の行列の1行目を表示する
print(tensor[:,0]) #4*4行列の1列目を表示

tensor[:,1]=0

print(tensor[1])
print(tensor[:,1])

print(tensor)

tensor[:,:]=3
print(tensor)

tensor[2,2]=9
print(tensor)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([1., 1., 1., 1.])
tensor([1., 1., 1., 1.])
tensor([1., 0., 1., 1.])
tensor([0., 0., 0., 0.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])
tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 9., 3.],
        [3., 3., 3., 3.]])


このように、特定の列・行の値を変えることもできるし、何行何列の要素だけ変更するということもできます

ここで、ふと気になるのは[:,1]という表現ですね。この[:]ってどういう意味でしょう?

In [13]:
tensor=torch.tensor([1,2,3])
print(tensor[:])

tensor([1, 2, 3])


この[:]は配列のある列や行の要素すべてのことを指します。上の例のように、tensorの値がすべて表示されていますね。これはよく出てくる表現なので覚えておきましょう。

続いて、tensor同士の結合をしてみましょう

In [14]:
tensor=torch.ones(3,4)

tensor[:,2]=4

print(tensor)

t2=torch.cat([tensor,tensor,tensor],dim=1)
print(t2)

t3=torch.cat([tensor,tensor,tensor],dim=0)
print(t3)

tensor([[1., 1., 4., 1.],
        [1., 1., 4., 1.],
        [1., 1., 4., 1.]])
tensor([[1., 1., 4., 1., 1., 1., 4., 1., 1., 1., 4., 1.],
        [1., 1., 4., 1., 1., 1., 4., 1., 1., 1., 4., 1.],
        [1., 1., 4., 1., 1., 1., 4., 1., 1., 1., 4., 1.]])
tensor([[1., 1., 4., 1.],
        [1., 1., 4., 1.],
        [1., 1., 4., 1.],
        [1., 1., 4., 1.],
        [1., 1., 4., 1.],
        [1., 1., 4., 1.],
        [1., 1., 4., 1.],
        [1., 1., 4., 1.],
        [1., 1., 4., 1.]])


このtorch.catは、tensorを配列に入れることで、そのtensor達をつなぎ合わせた新しいtensorを作ることができます。

4、tensorへの変換

pytorchにおいて、データを学習させるには、すべてのデータをtensor型に変換する必要がありました。しかし、学習データと呼ばれるものは、numpyで生成されたデータや、jpegといった画像データなどで、どれも変換が必要なものばかりです。

numpyの変換自体はとても簡単です。torchの機能として、「torch.from_numpy()」はその一つです

In [11]:
#まずはnumpyのインストールから。入ってる人でもスキップはされない
%pip install numpy

import numpy as np #numpyは慣用的にnpと略されて使用されます。

n=np.ones(5)
print(n)
print(type(n))

t=torch.from_numpy(n)
print(t)

Note: you may need to restart the kernel to use updated packages.
[1. 1. 1. 1. 1.]
<class 'numpy.ndarray'>
tensor([1., 1., 1., 1., 1.], dtype=torch.float64)



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


また、tensorからnumpyへの変換も可能です。

In [12]:
n2=t.numpy()
print(n2)
print(type(n2))

[1. 1. 1. 1. 1.]
<class 'numpy.ndarray'>


この両方向への変換はよく使うので覚えておくといいでしょう。しかも楽ですもんね。

次は画像です。こいつはtorchだけでは無理です。そこで、「pillow」と呼ばれる画像読み込みライブラリで読み込んだ画像をtensorに変換する「torchvision」を使います。

画像データ自体は「images」フォルダの「to_tensor.jpg」使います

In [13]:
#pillowをインストール
%pip install Pillow
import torchvision
from PIL import Image

#画像を読み込み
img=Image.open('images/to_tensor.jpg') #これは「相対パス」表記で示された画像を読み込んでいます。

#画像を変換
tensor_img=torchvision.transforms.functional.to_tensor(img)
print(tensor_img)

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])


一応これで変換はできていますが、ちょっとデータが多すぎて省略表示になっていますね。というのも1ピクセル単位で読み込んで変換してるので無理はないです。

そういえば、「相対パス」って習いましたか?　習っていないなら近いうちに説明を加えた演習を行いたいので、別途メールでもいいので知らせていただけると嬉しいです。

今回の演習は以上です。少し長かったですがお疲れ様でした。

In [14]:
#おまけ、pip の upgrade は定期的に
!python.exe -m pip install --upgrade pip

  Using cached pip-23.0.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.0
    Uninstalling pip-23.0:
      Successfully uninstalled pip-23.0
